In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

##Filters
A Filter is a function from an asset and a moment in time to a boolean:
```
F(asset, timestamp) -> boolean
```
In Pipeline, [Filters](https://www.quantopian.com/help#quantopian_pipeline_filters_Filter) are used for narrowing down the set of securities included in a computation or in the final output of a pipeline. There are two common ways to create a `Filter`: comparison operators and `Factor`/`Classifier` methods.

###Comparison Operators
Comparison operators on `Factors` and `Classifiers` produce Filters. Since we haven't looked at `Classifiers` yet, let's stick to examples using `Factors`. The following example produces a filter that returns `True` whenever the latest close price is above $20.

In [2]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

And this example produces a filter that returns True whenever the 10-day mean is below the 30-day mean.

In [3]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
mean_crossover_filter = mean_close_10 < mean_close_30

Remember, each security will get its own `True` or `False` value each day.

###Factor/Classifier Methods
Various methods of the `Factor` and `Classifier` classes return `Filters`. Again, since we haven't yet looked at `Classifiers`, let's stick to `Factor` methods for now (we'll look at `Classifier` methods later). The `Factor.top(n)` method produces a `Filter` that returns `True` for the top `n` securities of a given `Factor`. The following example produces a filter that returns `True` for exactly 200 securities every day, indicating that those securities were in the top 200 by last close price across all known securities.

In [5]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

For a full list of `Factor` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor).

For a full list of `Classifier` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier).

##Dollar Volume Filter
As a starting example, let's create a filter that returns `True` if a security's 30-day average dollar volume is above $10,000,000. To do this, we'll first need to create an `AverageDollarVolume` factor to compute the 30-day average dollar volume. Let's include the built-in `AverageDollarVolume` factor in our imports:

In [6]:
from quantopian.pipeline.factors import AverageDollarVolume

And then, let's instantiate our average dollar volume factor.

In [7]:
dollar_volume = AverageDollarVolume(window_length=30)

By default, `AverageDollarVolume` uses `USEquityPricing.close` and `USEquityPricing.volume` as its `inputs`, so we don't specify them.

Now that we have a dollar volume factor, we can create a filter with a boolean expression. The following line creates a filter returning `True` for securities with a `dollar_volume` greater than 10,000,000:

In [8]:
high_dollar_volume = (dollar_volume > 10000000)

To see what this filter looks like, let's can add it as a column to the pipeline we defined in the previous lesson.

In [9]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }
    )

If we make and run our pipeline, we now have a column `high_dollar_volume` with a boolean value corresponding to the result of the expression for each security.

In [10]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
result

high_dollar_volume  \
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                       True   
                          Equity(21 [AAME])                    False   
                          Equity(24 [AAPL])                     True   
                          Equity(25 [HWM_PR])                  False   
                          Equity(41 [ARCB])                    False   
                          Equity(52 [ABM])                     False   
                          Equity(53 [ABMD])                     True   
                          Equity(62 [ABT])                      True   
                          Equity(64 [GOLD])                     True   
                          Equity(66 [AB])                      False   
                          Equity(67 [ADSK])                     True   
                          Equity(70 [VBF])                     False   
                          Equity(76 [TAP])                      True   
                          Equity(100 [IEP])                    False   
                          Equity(106 [ACU])                    False   
                          Equity(110 [RAMP])                    True   
                          Equity(112 [ACY])                    False   
                          Equity(114 [ADBE])                    True   
                          Equity(117 [AEY])                    False   
                          Equity(122 [ADI])                     True   
                          Equity(128 [ADM])                     True   
                          Equity(149 [ADX])                    False   
                          Equity(153 [AE])                     False   
                          Equity(154 [AEM])                     True   
                          Equity(157 [AEG])                    False   
                          Equity(161 [AEP])                     True   
                          Equity(166 [AES])                     True   
                          Equity(185 [AFL])                     True   
                          Equity(192 [ATAX])                   False   
                          Equity(197 [AGCO])                    True   
...                                                              ...   
                          Equity(53097 [GNLN])                 False   
                          Equity(53098 [SHLL])                 False   
                          Equity(53099 [RYLD])                 False   
                          Equity(53100 [SHLL_WS])              False   
                          Equity(53101 [MREO])                 False   
                          Equity(53102 [SOHO_N])               False   
                          Equity(53103 [USI])                  False   
                          Equity(53104 [ALTG_WS])              False   
                          Equity(53105 [SEIX])                 False   
                          Equity(53106 [ALTG])                 False   
                          Equity(53107 [ACTT_U])               False   
                          Equity(53108 [ETP_PRE])              False   
                          Equity(53109 [WTRU])                 False   
                          Equity(53112 [WAFU])                 False   
                          Equity(53113 [TPLC])                 False   
                          Equity(53114 [TPHD])                 False   
                          Equity(53115 [SY])                   False   
                          Equity(53116 [BYND])                  True   
                          Equity(53117 [TRVI])                 False   
                          Equity(53118 [TMDX])                 False   
                          Equity(53119 [KEY_PRK])              False   
                          Equity(53120 [YJ])                   False   
                          Equity(53121 [BUL])                  False   
                          Equity(53122 [RRBI])                 False   
                  

##Applying a Screen
By default, a pipeline produces computed values each day for every asset in the Quantopian database. Very often however, we only care about a subset of securities that meet specific criteria (for example, we might only care about securities that have enough daily trading volume to fill our orders quickly). We can tell our Pipeline to ignore securities for which a filter produces `False` by passing that filter to our Pipeline via the `screen` keyword.

To screen our pipeline output for securities with a 30-day average dollar volume greater than $10,000,000, we can simply pass our `high_dollar_volume` filter as the `screen` argument. This is what our `make_pipeline` function now looks like:

In [11]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume > 10000000

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=high_dollar_volume
    )

When we run this, the pipeline output only includes securities that pass the `high_dollar_volume` filter on a given day. For example, running this pipeline on May 5th, 2015 results in an output for ~2,100 securities

In [12]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 2079


percent_difference
2019-05-06 00:00:00+00:00 Equity(2 [HWM])                 0.045815
                          Equity(24 [AAPL])               0.036887
                          Equity(53 [ABMD])              -0.025557
                          Equity(62 [ABT])               -0.001032
                          Equity(64 [GOLD])              -0.044003
                          Equity(67 [ADSK])               0.044820
                          Equity(76 [TAP])                0.023185
                          Equity(110 [RAMP])              0.033836
                          Equity(114 [ADBE])              0.030868
                          Equity(122 [ADI])               0.031932
                          Equity(128 [ADM])               0.002744
                          Equity(154 [AEM])              -0.036071
                          Equity(161 [AEP])               0.005391
                          Equity(166 [AES])              -0.024324
                          Equity(185 [AFL])               0.003568
                          Equity(197 [AGCO])              0.013061
                          Equity(216 [HES])               0.023028
                          Equity(239 [AIG])               0.028965
                          Equity(266 [AJG])               0.028962
                          Equity(300 [ALK])               0.036741
                          Equity(301 [ALKS])             -0.081406
                          Equity(337 [AMAT])              0.037732
                          Equity(351 [AMD])               0.017213
                          Equity(353 [AME])               0.026951
                          Equity(368 [AMGN])             -0.039630
                          Equity(371 [TVTY])              0.096734
                          Equity(410 [AN])                0.074171
                          Equity(438 [AON])               0.023199
                          Equity(448 [APA])              -0.030744
                          Equity(455 [APC])               0.244091
...                                                            ...
                          Equity(52511 [LTHM])           -0.072518
                          Equity(52517 [PLAN])            0.013254
                          Equity(52525 [REZI])            0.078479
                          Equity(52537 [STNE])           -0.216606
                          Equity(52571 [YETI])            0.050781
                          Equity(52592 [LIN])             0.001633
                          Equity(52594 [ETRN])            0.004603
                          Equity(52603 [APHA])           -0.144202
                          Equity(52709 [TME])            -0.019613
                          Equity(52735 [RBZ])            -0.258347
                          Equity(52747 [DELL])            0.057653
                          Equity(52751 [MRNA])            0.088434
                          Equity(52812 [HEXO])            0.103957
                          Equity(52838 [CVET])            0.005165
                          Equity(52912 [VFF])            -0.113452
                          Equity(52917 [CTST])           -0.088385
                          Equity(52954 [SWAV])            0.139249
                          Equity(52968 [FOXA])            0.017403
                          Equity(52969 [FOX])             0.020950
                          Equity(52991 [DOW])             0.009031
                          Equity(52993 [TIGR])           -0.020752
                          Equity(52998 [LEVI])            0.001276
                          Equity(53023 [LYFT])           -0.037528
                          Equity(53046 [TW])             -0.000798
                          Equity(53056 [ALC])             0.011070
                          Equity(53063 [JMIA])            0.016815
                          Equity(53064 [PD])              0.029497
                          Equity(53089 [PINS])            0.000000
                          Eq

##Inverting a Filter
The `~` operator is used to invert a filter, swapping all `True` values with `Falses` and vice-versa. For example, we can write the following to filter for low dollar volume securities:

In [13]:
low_dollar_volume = ~high_dollar_volume

This will return `True` for all securities with an average dollar volume below or equal to $10,000,000 over the last 30 days.

In the next lesson, we will look at combining filters.